In [ ]:
%matplotlib inline
from decimal import Decimal
import itertools
from pprint import pprint
import math

import pandas
import seaborn
from matplotlib import pyplot as plt
plt.style.use('seaborn-poster')
import numpy

import metrics
import simulate
import harvesting
import market
import plot
import mortality
from plot import plt
import withdrawal
from portfolio import Portfolio
import montecarlo

In [ ]:
series = market.Returns_US_1871()
START_YEAR=1871

#series = market.Japan_1957()
#START_YEAR=1957

In [ ]:
def withdrawals(series,
                strategy_pair,
                portfolio=(600000, 400000),
                years=40):
    withdraw, harvesting = strategy_pair
    portfolio = Portfolio(portfolio[0], portfolio[1])
    strategy = harvesting(portfolio).harvest()
    strategy.send(None)
    withdrawal_strategy = withdraw(portfolio, strategy).withdrawals()
    annual = []

    # Withdrawals happen at the start of the year, so the first time
    # we don't have any performance data to send them....
    data = withdrawal_strategy.send(None)
    annual.append(data)
    years -= 1

    for _, d in zip(range(years), series):
        data = withdrawal_strategy.send(d)
        annual.append(data)
    return annual


In [ ]:
def test_withdrawal_strategies(year, strategies_to_test, years=25):
    results = {}
    for strategy in strategies_to_test:
        starting_portfolio = (600000,400000)

        x = withdrawals(series.iter_from(year),
                                 years=years,
                                 strategy_pair=strategy,
                                 portfolio=starting_portfolio)
        name = strategy[0].__name__
        results[name] = x
    return results

In [ ]:
def semideviation(frame, goal=40000):
    values = frame[lambda s: s < goal]
    if len(values) > 0:
        sumvalues = sum(((goal - v) ** 2 for v in values))
        average = sumvalues / len(values)
        return math.sqrt(average)
    else:
        return 0

In [ ]:
def median_cut(frame, goal=40000):
    values = frame[lambda s: s < goal]
    if len(values) > 0:
        return values.apply(lambda s: s - goal).median()
    else:
        return 0

In [ ]:
lens = lambda x: float(x.withdraw_r)

In [ ]:
strategies_to_test = [
    (withdrawal.VPW, harvesting.N_60_RebalanceHarvesting),
    (withdrawal.VPWBank, harvesting.N_60_RebalanceHarvesting),
]

data = {
    'VPW' : [],
    'VPWBank' : []
}

In [ ]:
%pdb on

for year in range(START_YEAR, 1985):
    results = test_withdrawal_strategies(year, strategies_to_test, years=30)
    for strategy in results.keys():
        ws = [lens(n) for n in results[strategy]]
        data[strategy] = data[strategy] + ws

for strategy, values in data.items():
    s = pandas.Series(values)
    print('Semideviation', strategy, semideviation(s))
    
for strategy, values in data.items():
    s = pandas.Series(values)
    print('Stdev', strategy, s.std())
    
for strategy, values in data.items():
    s = pandas.Series(values)
    print('Median Cut', strategy, median_cut(s))

In [ ]:
num_vpw_cuts = 0
num_bank_cuts = 0
for year in range(START_YEAR, 1985):
    results = test_withdrawal_strategies(year, strategies_to_test, years=30)
    
    vpw_real_wd = pandas.Series([lens(n) for n in results['VPW']])
    bank_real_wd = pandas.Series([lens(n) for n in results['VPWBank']])
    vpw_cuts = len(vpw_real_wd[lambda s: s < 49000])
    bank_cuts = len(bank_real_wd[lambda s: s < 49000])
    if vpw_cuts != bank_cuts:
#    if vpw_cuts or bank_cuts:
        print(year, ':', 'VPW', vpw_cuts, 'Bank', bank_cuts)
        
    num_vpw_cuts += vpw_cuts
    num_bank_cuts += bank_cuts
            
print('Num VPW Cuts', num_vpw_cuts)
print('Num Bank Cuts', num_bank_cuts)

In [ ]:
def compare_year(year, lens, title):
    results = test_withdrawal_strategies(year, strategies_to_test, years=30)
    
    fig, ax = plt.subplots()

    if '%' not in title:
        plot.format_axis_labels_with_commas(ax.get_yaxis())

    plt.xlabel('Year of Retirement')
    plt.title('Retirement in Year %s (%s)' % (year, title))

    for strategy in (sorted(results.keys())):
        ax_n = fig.add_subplot(111, sharex=ax, sharey=ax)
        ws = [lens(n) for n in results[strategy]]
        ax_n.plot(ws, label=strategy)
        ax_n.set_ymargin(0.05)
    plt.legend(loc=0)
    ax.set_ylim(bottom=0)
    plt.show()

In [ ]:
def chart_all(year):
#    compare_year(year, lambda x: x.portfolio_stocks/x.portfolio_n*100, "Stock %")
#    compare_year(year, lambda x: x.portfolio_n, "Portfolio (Nominal)")
#    compare_year(year, lambda x: x.portfolio_r, "Portfolio (Real)")
#    compare_year(year, lambda x: x.withdraw_n, "Withdrawals (Nominal)")
    compare_year(year, lambda x: x.withdraw_r, "Withdrawals (Real)")

In [ ]:
chart_all(1897)